In [56]:
import sys
import os
import copy
import torch
import torch.nn as nn
from typing import List, Any
import pytorch_lightning as pl
from datetime import datetime
from torchvision.models import resnet18

In [7]:
# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../../../')
sys.path.append(NOMEROFF_NET_DIR)

In [17]:
from nomeroff_net.tools.ocr_tools import plot_loss, print_prediction
from nomeroff_net.tools.mcm import get_device_torch
from nomeroff_net.tools.ocr_tools import (StrLabelConverter,
                                          decode_prediction,
                                          decode_batch)
from nomeroff_net.nnmodels.ocr_model import NPOcrNet, weights_init

In [9]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

# model from branch v3.2
https://github.com/ria-com/nomeroff-net/blob/v3.2/nomeroff_net/nnmodels/ocr_model.py

In [21]:
class BlockRNNv3_2(nn.Module):
    def __init__(self, in_size, hidden_size, out_size, bidirectional):
        super(BlockRNNv3_2, self).__init__()
        self.in_size = in_size
        self.hidden_size = hidden_size
        self.out_size = out_size
        self.bidirectional = bidirectional
        # layers
        self.gru = nn.LSTM(in_size, hidden_size, bidirectional=bidirectional, batch_first=True)

    def forward(self, batch, add_output=False):
        """
        in array:
            batch - [seq_len , batch_size, in_size]
        out array:
            out - [seq_len , batch_size, out_size]
        """
        outputs, hidden = self.gru(batch)
        out_size = int(outputs.size(2) / 2)
        if add_output:
            outputs = outputs[:, :, :out_size] + outputs[:, :, out_size:]
        return outputs


class NPOcrNetv3_2(pl.LightningModule):
    def __init__(self,
                 letters: List = None,
                 letters_max: int = 0,
                 max_plate_length: int = 8,
                 learning_rate: float = 0.02,
                 hidden_size: int = 32,
                 bidirectional: bool = True,
                 label_converter: Any = None,
                 val_dataset: Any = None,
                 weight_decay: float = 1e-5,
                 momentum: float = 0.9,
                 clip_norm: int = 5):
        super().__init__()
        self.save_hyperparameters()

        self.letters = letters
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.clip_norm = clip_norm
        self.momentum = momentum
        self.max_plate_length = max_plate_length

        self.hidden_size = hidden_size
        self.bidirectional = bidirectional

        self.label_converter = label_converter
        
        # convolutions 
        resnet = resnet18(pretrained=True)
        modules = list(resnet.children())[:-3]
        self.resnet = nn.Sequential(*modules)

        # RNN + Linear
        self.linear1 = nn.Linear(1024, 512)
        self.gru1 = BlockRNNv3_2(512, hidden_size, hidden_size,
                             bidirectional=bidirectional)
        self.gru2 = BlockRNNv3_2(hidden_size, hidden_size, letters_max,
                             bidirectional=bidirectional)
        self.linear2 = nn.Linear(hidden_size * 2, letters_max)

        self.automatic_optimization = True
        self.criterion = None
        self.val_dataset = val_dataset
        self.train_losses = []
        self.val_losses = []

    def forward(self, batch: torch.float64):
        batch_size = batch.size(0)
        
        # convolutions
        batch = self.resnet(batch)

        # make sequences of image features
        batch = batch.permute(0, 3, 1, 2)
        n_channels = batch.size(1)
        batch = batch.reshape(batch_size, n_channels, -1)

        batch = self.linear1(batch)

        # rnn layers
        batch = self.gru1(batch, add_output=True)
        batch = self.gru2(batch)
        # output
        batch = self.linear2(batch)
        batch = batch.permute(1, 0, 2)
        return batch

    def init_loss(self):
        self.criterion = nn.CTCLoss(blank=0, zero_infinity=True, reduction='mean')

    def calculate_loss(self, logits, texts):
        if self.criterion is None:
            self.init_loss()

        # get infomation from prediction
        device = logits.device
        input_len, batch_size, vocab_size = logits.size()
        # encode inputs
        logits = logits.log_softmax(2)
        encoded_texts, text_lens = self.label_converter.encode(texts)
        logits_lens = torch.full(size=(batch_size,), fill_value=input_len, dtype=torch.int32)
        # calculate ctc
        loss = self.criterion(
            logits,
            encoded_texts,
            logits_lens.to(device),
            text_lens)
        return loss

    def step(self, batch):

        x, texts = batch

        output = self.forward(x)

        loss = self.calculate_loss(output, texts)
        return loss

    def on_save_checkpoint(self, _):
        if self.current_epoch and self.val_dataset:
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            print_prediction(self, self.val_dataset, device, self.label_converter)
            plot_loss(self.current_epoch, self.train_losses, self.val_losses)

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=self.learning_rate,
            nesterov=True,
            weight_decay=self.weight_decay,
            momentum=self.momentum)
        return optimizer

    def training_step(self, batch, batch_idx):
        loss = self.step(batch)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        tqdm_dict = {
            'train_loss': loss,
        }
        return {
            'loss': loss,
            'progress_bar': tqdm_dict,
            'log': tqdm_dict
        }

    def validation_step(self, batch, batch_idx):
        loss = self.step(batch)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        tqdm_dict = {
            'val_loss': loss,
        }
        return {
            'val_loss': loss,
            'progress_bar': tqdm_dict,
            'log': tqdm_dict
        }

    def test_step(self, batch, batch_idx):
        loss = self.step(batch)
        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        tqdm_dict = {
            'test_loss': loss,
        }
        return {
            'test_loss': loss,
            'progress_bar': tqdm_dict,
            'log': tqdm_dict
        }

# model from branch v3.3
https://github.com/ria-com/nomeroff-net/blob/v3.3/nomeroff_net/nnmodels/ocr_model.py

In [22]:
class BlockRNNv3_3(nn.Module):
    def __init__(self, in_size, hidden_size, out_size, bidirectional, recurrent_nn=nn.LSTM):
        super().__init__()
        self.in_size = in_size
        self.hidden_size = hidden_size
        self.out_size = out_size
        self.bidirectional = bidirectional

        # layers
        self.rnn = recurrent_nn(in_size, hidden_size, bidirectional=bidirectional, batch_first=True)

    def forward(self, batch, add_output=False):
        """
        in array:
            batch - [seq_len , batch_size, in_size]
        out array:
            out - [seq_len , batch_size, out_size]
        """
        outputs, hidden = self.rnn(batch)
        out_size = int(outputs.size(2) / 2)
        if add_output:
            outputs = outputs[:, :, :out_size] + outputs[:, :, out_size:]
        return outputs


class NPOcrNetv3_3(pl.LightningModule):
    def __init__(self,
                 letters: List = None,
                 letters_max: int = 0,
                 max_text_len: int = 8,
                 learning_rate: float = 0.02,
                 height: int = 50,
                 width: int = 200,
                 color_channels: int = 3,
                 bidirectional: bool = True,
                 label_converter: Any = None,
                 val_dataset: Any = None,
                 weight_decay: float = 1e-5,
                 momentum: float = 0.9,
                 clip_norm: int = 5,
                 hidden_size=32,
                 linear_size=512,
                 backbone=None):
        print(backbone, linear_size, hidden_size, height, width)
        super().__init__()
        self.save_hyperparameters()

        self.width = width
        self.height = height
        self.linear_size = linear_size
        self.color_channels = color_channels

        self.letters = letters
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.clip_norm = clip_norm
        self.momentum = momentum
        self.max_text_len = max_text_len

        self.hidden_size = hidden_size
        self.bidirectional = bidirectional

        self.label_converter = label_converter

        # convolutions
        if backbone is None:
            backbone = resnet18
        conv_nn = backbone(pretrained=True)
        if 'resnet' in str(backbone):
            conv_modules = list(conv_nn.children())[:-3]
        elif 'efficientnet' in str(backbone):
            conv_modules = list(conv_nn.children())[:-2]
        elif 'shufflenet' in str(backbone):
            conv_modules = list(conv_nn.children())[:-3]
        else:
            raise NotImplementedError(backbone)
        self.conv_nn = nn.Sequential(*conv_modules)
        _, backbone_c, backbone_h, backbone_w = self.conv_nn(torch.rand((1, color_channels, height, width))).shape

        assert backbone_w > max_text_len

        # RNN + Linear
        self.linear1 = nn.Linear(backbone_c*backbone_h, self.linear_size)
        self.recurrent_layer1 = BlockRNNv3_3(self.linear_size, hidden_size, hidden_size,
                                         bidirectional=bidirectional)
        self.recurrent_layer2 = BlockRNNv3_3(hidden_size, hidden_size, letters_max,
                                         bidirectional=bidirectional)

        self.linear2 = nn.Linear(hidden_size * 2, letters_max)

        self.automatic_optimization = True
        self.criterion = None
        self.val_dataset = val_dataset
        self.train_losses = []
        self.val_losses = []

    def forward(self, batch: torch.float64):
        """
        forward
        """
        batch_size = batch.size(0)
        
        # convolutions
        batch = self.conv_nn(batch)

        # make sequences of image features
        batch = batch.permute(0, 3, 1, 2)
        n_channels = batch.size(1)
        batch = batch.reshape(batch_size, n_channels, -1)
        batch = self.linear1(batch)

        # rnn layers
        batch = self.recurrent_layer1(batch, add_output=True)
        batch = self.recurrent_layer2(batch)

        # output
        batch = self.linear2(batch)
        batch = batch.permute(1, 0, 2)
        return batch

    def init_loss(self):
        self.criterion = nn.CTCLoss(blank=0, zero_infinity=True, reduction='mean')

    def calculate_loss(self, logits, texts):
        if self.criterion is None:
            self.init_loss()

        # get infomation from prediction
        device = logits.device
        input_len, batch_size, vocab_size = logits.size()
        # encode inputs
        logits = logits.log_softmax(2)
        encoded_texts, text_lens = self.label_converter.encode(texts)
        logits_lens = torch.full(size=(batch_size,), fill_value=input_len, dtype=torch.int32)
        # calculate ctc
        loss = self.criterion(
            logits,
            encoded_texts,
            logits_lens.to(device),
            text_lens)
        return loss

    def step(self, batch):
        x, texts = batch
        output = self.forward(x)
        loss = self.calculate_loss(output, texts)
        return loss

    def on_save_checkpoint(self, _):
        if self.current_epoch and self.val_dataset:
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            print_prediction(self, self.val_dataset, device, self.label_converter)
            plot_loss(self.current_epoch, self.train_losses, self.val_losses)

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=self.learning_rate,
            nesterov=True,
            weight_decay=self.weight_decay,
            momentum=self.momentum)
        return optimizer

    def training_step(self, batch, batch_idx):
        loss = self.step(batch)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        tqdm_dict = {
            'train_loss': loss,
        }
        return {
            'loss': loss,
            'progress_bar': tqdm_dict,
            'log': tqdm_dict
        }

    def validation_step(self, batch, batch_idx):
        loss = self.step(batch)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        tqdm_dict = {
            'val_loss': loss,
        }
        return {
            'val_loss': loss,
            'progress_bar': tqdm_dict,
            'log': tqdm_dict
        }

    def test_step(self, batch, batch_idx):
        loss = self.step(batch)
        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        tqdm_dict = {
            'test_loss': loss,
        }
        return {
            'test_loss': loss,
            'progress_bar': tqdm_dict,
            'log': tqdm_dict
        }

# Convert chekpoint keys v3.2 -> v3.3

## am

In [58]:
DATASET_NAME = "am"
VERSION = f"{datetime.now().strftime('%Y_%m_%d')}"

RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, 
                                 "data/models/", 
                                 'anpr_ocr_{}_{}.ckpt'.format(DATASET_NAME, VERSION))
letters = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H',
           'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

max_plate_length = 7
label_converter = StrLabelConverter("".join(letters), max_plate_length)

modelv3_2 = NPOcrNetv3_2.load_from_checkpoint("https://nomeroff.net.ua/models/ocr/am/torch/model_v3.1/anpr_ocr_am_2022_03_28_pytorch_lightning.ckpt",
                                              map_location=torch.device('cpu'),
                                              letters=letters,
                                              letters_max=len(letters) + 1,
                                              label_converter=label_converter,
                                              max_plate_length=max_plate_length)

modelv3_3 = NPOcrNetv3_3(letters,
                         letters_max=len(letters) + 1,
                         label_converter=label_converter,
                         max_text_len=max_plate_length)

state_dict_v3_2 = modelv3_2.state_dict()
state_dict_v3_3 = copy.deepcopy(state_dict_v3_2)
for key in state_dict_v3_2:
    if 'resnet' in key:
        key_v3_3 = key.replace('resnet', 'conv_nn')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
    elif 'gru' in key:
        key_v3_3 = key.replace('.gru.', '.rnn.')
        key_v3_3 = key_v3_3.replace('gru', 'recurrent_layer')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
        
modelv3_3.load_state_dict(state_dict_v3_3)
torch.save({"state_dict": modelv3_3.state_dict()}, RESULT_MODEL_PATH)

None 512 32 50 200


## by

In [59]:
DATASET_NAME = "by"
VERSION = f"{datetime.now().strftime('%Y_%m_%d')}"

RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, 
                                 "data/models/", 
                                 'anpr_ocr_{}_{}.ckpt'.format(DATASET_NAME, VERSION))
letters = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'E', 'H', 'I',
           'K', 'M', 'O', 'P', 'T', 'X']

max_plate_length = 7
label_converter = StrLabelConverter("".join(letters), max_plate_length)

modelv3_2 = NPOcrNetv3_2.load_from_checkpoint("https://nomeroff.net.ua/models/ocr/by/torch/model_v3.1/anpr_ocr_by_2022_03_28_pytorch_lightning.ckpt",
                                              map_location=torch.device('cpu'),
                                              letters=letters,
                                              letters_max=len(letters) + 1,
                                              label_converter=label_converter,
                                              max_plate_length=max_plate_length)

modelv3_3 = NPOcrNetv3_3(letters,
                         letters_max=len(letters) + 1,
                         label_converter=label_converter,
                         max_text_len=max_plate_length)

state_dict_v3_2 = modelv3_2.state_dict()
state_dict_v3_3 = copy.deepcopy(state_dict_v3_2)
for key in state_dict_v3_2:
    if 'resnet' in key:
        key_v3_3 = key.replace('resnet', 'conv_nn')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
    elif 'gru' in key:
        key_v3_3 = key.replace('.gru.', '.rnn.')
        key_v3_3 = key_v3_3.replace('gru', 'recurrent_layer')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
        
modelv3_3.load_state_dict(state_dict_v3_3)
torch.save({"state_dict": modelv3_3.state_dict()}, RESULT_MODEL_PATH)

Downloading: "https://nomeroff.net.ua/models/ocr/by/torch/model_v3.1/anpr_ocr_by_2022_03_28_pytorch_lightning.ckpt" to /home/dmitroprobachay/.cache/torch/hub/checkpoints/anpr_ocr_by_2022_03_28_pytorch_lightning.ckpt


  0%|          | 0.00/13.3M [00:00<?, ?B/s]

None 512 32 50 200


## eu

In [60]:
DATASET_NAME = "eu"
VERSION = f"{datetime.now().strftime('%Y_%m_%d')}"

RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, 
                                 "data/models/", 
                                 'anpr_ocr_{}_{}.ckpt'.format(DATASET_NAME, VERSION))
letters = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "D", "E", "F", "G", "H", "I",
           "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

max_plate_length = 9
label_converter = StrLabelConverter("".join(letters), max_plate_length)

modelv3_2 = NPOcrNetv3_2.load_from_checkpoint("https://nomeroff.net.ua/models/ocr/eu/torch/model_v3.1/anpr_ocr_eu_2022_03_28_pytorch_lightning.ckpt",
                                              map_location=torch.device('cpu'),
                                              letters=letters,
                                              letters_max=len(letters) + 1,
                                              label_converter=label_converter,
                                              max_plate_length=max_plate_length)

modelv3_3 = NPOcrNetv3_3(letters,
                         letters_max=len(letters) + 1,
                         label_converter=label_converter,
                         max_text_len=max_plate_length)

state_dict_v3_2 = modelv3_2.state_dict()
state_dict_v3_3 = copy.deepcopy(state_dict_v3_2)
for key in state_dict_v3_2:
    if 'resnet' in key:
        key_v3_3 = key.replace('resnet', 'conv_nn')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
    elif 'gru' in key:
        key_v3_3 = key.replace('.gru.', '.rnn.')
        key_v3_3 = key_v3_3.replace('gru', 'recurrent_layer')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
        
modelv3_3.load_state_dict(state_dict_v3_3)
torch.save({"state_dict": modelv3_3.state_dict()}, RESULT_MODEL_PATH)

Downloading: "https://nomeroff.net.ua/models/ocr/eu/torch/model_v3.1/anpr_ocr_eu_2022_03_28_pytorch_lightning.ckpt" to /home/dmitroprobachay/.cache/torch/hub/checkpoints/anpr_ocr_eu_2022_03_28_pytorch_lightning.ckpt


  0%|          | 0.00/13.3M [00:00<?, ?B/s]

None 512 32 50 200


## eu_ua_1995

In [61]:
DATASET_NAME = "eu_ua_1995"
VERSION = f"{datetime.now().strftime('%Y_%m_%d')}"

RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, 
                                 "data/models/", 
                                 'anpr_ocr_{}_{}.ckpt'.format(DATASET_NAME, VERSION))
letters = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'E', 'H', 'I', 'K', 'M',
           'O', 'P', 'T', 'X']

max_plate_length = 8
label_converter = StrLabelConverter("".join(letters), max_plate_length)

modelv3_2 = NPOcrNetv3_2.load_from_checkpoint("https://nomeroff.net.ua/models/ocr/ua-1995/torch/model_v3.1/anpr_ocr_eu_ua_1995_2022_03_28_pytorch_lightning.ckpt",
                                              map_location=torch.device('cpu'),
                                              letters=letters,
                                              letters_max=len(letters) + 1,
                                              label_converter=label_converter,
                                              max_plate_length=max_plate_length)

modelv3_3 = NPOcrNetv3_3(letters,
                         letters_max=len(letters) + 1,
                         label_converter=label_converter,
                         max_text_len=max_plate_length)

state_dict_v3_2 = modelv3_2.state_dict()
state_dict_v3_3 = copy.deepcopy(state_dict_v3_2)
for key in state_dict_v3_2:
    if 'resnet' in key:
        key_v3_3 = key.replace('resnet', 'conv_nn')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
    elif 'gru' in key:
        key_v3_3 = key.replace('.gru.', '.rnn.')
        key_v3_3 = key_v3_3.replace('gru', 'recurrent_layer')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
        
modelv3_3.load_state_dict(state_dict_v3_3)
torch.save({"state_dict": modelv3_3.state_dict()}, RESULT_MODEL_PATH)

Downloading: "https://nomeroff.net.ua/models/ocr/ua-1995/torch/model_v3.1/anpr_ocr_eu_ua_1995_2022_03_28_pytorch_lightning.ckpt" to /home/dmitroprobachay/.cache/torch/hub/checkpoints/anpr_ocr_eu_ua_1995_2022_03_28_pytorch_lightning.ckpt


  0%|          | 0.00/13.3M [00:00<?, ?B/s]

None 512 32 50 200


## eu_ua_2004_2015

In [62]:
DATASET_NAME = "eu_ua_2004_2015"
VERSION = f"{datetime.now().strftime('%Y_%m_%d')}"

RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, 
                                 "data/models/", 
                                 'anpr_ocr_{}_{}.ckpt'.format(DATASET_NAME, VERSION))
letters = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "D", "E", "F", "G", "H", 
           "I", "K", "M", "O", "P", "T", "X", "Y", "Z"]

max_plate_length = 8
label_converter = StrLabelConverter("".join(letters), max_plate_length)

modelv3_2 = NPOcrNetv3_2.load_from_checkpoint("https://nomeroff.net.ua/models/ocr/ua/torch/model_v3.1/anpr_ocr_eu_2004_2015_2022_10_29_pytorch_lightning.ckpt",
                                              map_location=torch.device('cpu'),
                                              letters=letters,
                                              letters_max=len(letters) + 1,
                                              label_converter=label_converter,
                                              max_plate_length=max_plate_length)

modelv3_3 = NPOcrNetv3_3(letters,
                         letters_max=len(letters) + 1,
                         label_converter=label_converter,
                         max_text_len=max_plate_length)

state_dict_v3_2 = modelv3_2.state_dict()
state_dict_v3_3 = copy.deepcopy(state_dict_v3_2)
for key in state_dict_v3_2:
    if 'resnet' in key:
        key_v3_3 = key.replace('resnet', 'conv_nn')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
    elif 'gru' in key:
        key_v3_3 = key.replace('.gru.', '.rnn.')
        key_v3_3 = key_v3_3.replace('gru', 'recurrent_layer')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
        
modelv3_3.load_state_dict(state_dict_v3_3)
torch.save({"state_dict": modelv3_3.state_dict()}, RESULT_MODEL_PATH)

Downloading: "https://nomeroff.net.ua/models/ocr/ua/torch/model_v3.1/anpr_ocr_eu_2004_2015_2022_10_29_pytorch_lightning.ckpt" to /home/dmitroprobachay/.cache/torch/hub/checkpoints/anpr_ocr_eu_2004_2015_2022_10_29_pytorch_lightning.ckpt


  0%|          | 0.00/26.5M [00:00<?, ?B/s]

None 512 32 50 200


## ge

In [64]:
DATASET_NAME = "ge"
VERSION = f"{datetime.now().strftime('%Y_%m_%d')}"

RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, 
                                 "data/models/", 
                                 'anpr_ocr_{}_{}.ckpt'.format(DATASET_NAME, VERSION))
letters = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "D", "E", "F", "G", "H", "I",
           "J", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

max_plate_length = 8
label_converter = StrLabelConverter("".join(letters), max_plate_length)

modelv3_2 = NPOcrNetv3_2.load_from_checkpoint("https://nomeroff.net.ua/models/ocr/ge/torch/model_v3.1/anpr_ocr_ge_2022_03_28_pytorch_lightning.ckpt",
                                              map_location=torch.device('cpu'),
                                              letters=letters,
                                              letters_max=len(letters) + 1,
                                              label_converter=label_converter,
                                              max_plate_length=max_plate_length)

modelv3_3 = NPOcrNetv3_3(letters,
                         letters_max=len(letters) + 1,
                         label_converter=label_converter,
                         max_text_len=max_plate_length)

state_dict_v3_2 = modelv3_2.state_dict()
state_dict_v3_3 = copy.deepcopy(state_dict_v3_2)
for key in state_dict_v3_2:
    if 'resnet' in key:
        key_v3_3 = key.replace('resnet', 'conv_nn')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
    elif 'gru' in key:
        key_v3_3 = key.replace('.gru.', '.rnn.')
        key_v3_3 = key_v3_3.replace('gru', 'recurrent_layer')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
        
modelv3_3.load_state_dict(state_dict_v3_3)
torch.save({"state_dict": modelv3_3.state_dict()}, RESULT_MODEL_PATH)

Downloading: "https://nomeroff.net.ua/models/ocr/ge/torch/model_v3.1/anpr_ocr_ge_2022_03_28_pytorch_lightning.ckpt" to /home/dmitroprobachay/.cache/torch/hub/checkpoints/anpr_ocr_ge_2022_03_28_pytorch_lightning.ckpt


  0%|          | 0.00/13.3M [00:00<?, ?B/s]

None 512 32 50 200


## kg

In [65]:
DATASET_NAME = "kg"
VERSION = f"{datetime.now().strftime('%Y_%m_%d')}"

RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, 
                                 "data/models/", 
                                 'anpr_ocr_{}_{}.ckpt'.format(DATASET_NAME, VERSION))
letters = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9",
           "A", "B", "C", "D", "E", "F", "G", "H", "I", "J",
           "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T",
           "U", "V", "W", "X", "Y", "Z"]

max_plate_length = 8
label_converter = StrLabelConverter("".join(letters), max_plate_length)

modelv3_2 = NPOcrNetv3_2.load_from_checkpoint("https://nomeroff.net.ua/models/ocr/kg/torch/model_v3.1/anpr_ocr_kg_2022_03_28_pytorch_lightning.ckpt",
                                              map_location=torch.device('cpu'),
                                              letters=letters,
                                              letters_max=len(letters) + 1,
                                              label_converter=label_converter,
                                              max_plate_length=max_plate_length)

modelv3_3 = NPOcrNetv3_3(letters,
                         letters_max=len(letters) + 1,
                         label_converter=label_converter,
                         max_text_len=max_plate_length)

state_dict_v3_2 = modelv3_2.state_dict()
state_dict_v3_3 = copy.deepcopy(state_dict_v3_2)
for key in state_dict_v3_2:
    if 'resnet' in key:
        key_v3_3 = key.replace('resnet', 'conv_nn')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
    elif 'gru' in key:
        key_v3_3 = key.replace('.gru.', '.rnn.')
        key_v3_3 = key_v3_3.replace('gru', 'recurrent_layer')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
        
modelv3_3.load_state_dict(state_dict_v3_3)
torch.save({"state_dict": modelv3_3.state_dict()}, RESULT_MODEL_PATH)

Downloading: "https://nomeroff.net.ua/models/ocr/kg/torch/model_v3.1/anpr_ocr_kg_2022_03_28_pytorch_lightning.ckpt" to /home/dmitroprobachay/.cache/torch/hub/checkpoints/anpr_ocr_kg_2022_03_28_pytorch_lightning.ckpt


  0%|          | 0.00/13.3M [00:00<?, ?B/s]

None 512 32 50 200


## kz

In [66]:
DATASET_NAME = "kz"
VERSION = f"{datetime.now().strftime('%Y_%m_%d')}"

RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, 
                                 "data/models/", 
                                 'anpr_ocr_{}_{}.ckpt'.format(DATASET_NAME, VERSION))
letters = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "D", "E", "F", "G", "H",
           "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

max_plate_length = 6
label_converter = StrLabelConverter("".join(letters), max_plate_length)

modelv3_2 = NPOcrNetv3_2.load_from_checkpoint("https://nomeroff.net.ua/models/ocr/kz/torch/model_v3.1/anpr_ocr_kz_2022_03_28_pytorch_lightning.ckpt",
                                              map_location=torch.device('cpu'),
                                              letters=letters,
                                              letters_max=len(letters) + 1,
                                              label_converter=label_converter,
                                              max_plate_length=max_plate_length)

modelv3_3 = NPOcrNetv3_3(letters,
                         letters_max=len(letters) + 1,
                         label_converter=label_converter,
                         max_text_len=max_plate_length)

state_dict_v3_2 = modelv3_2.state_dict()
state_dict_v3_3 = copy.deepcopy(state_dict_v3_2)
for key in state_dict_v3_2:
    if 'resnet' in key:
        key_v3_3 = key.replace('resnet', 'conv_nn')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
    elif 'gru' in key:
        key_v3_3 = key.replace('.gru.', '.rnn.')
        key_v3_3 = key_v3_3.replace('gru', 'recurrent_layer')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
        
modelv3_3.load_state_dict(state_dict_v3_3)
torch.save({"state_dict": modelv3_3.state_dict()}, RESULT_MODEL_PATH)

Downloading: "https://nomeroff.net.ua/models/ocr/kz/torch/model_v3.1/anpr_ocr_kz_2022_03_28_pytorch_lightning.ckpt" to /home/dmitroprobachay/.cache/torch/hub/checkpoints/anpr_ocr_kz_2022_03_28_pytorch_lightning.ckpt


  0%|          | 0.00/13.3M [00:00<?, ?B/s]

None 512 32 50 200


## md

In [67]:
DATASET_NAME = "md"
VERSION = f"{datetime.now().strftime('%Y_%m_%d')}"

RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, 
                                 "data/models/", 
                                 'anpr_ocr_{}_{}.ckpt'.format(DATASET_NAME, VERSION))
letters = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "D", "E", "F", "G", "H", "I",
           "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

max_plate_length = 7
label_converter = StrLabelConverter("".join(letters), max_plate_length)

modelv3_2 = NPOcrNetv3_2.load_from_checkpoint("https://nomeroff.net.ua/models/ocr/md/torch/model_v3.1/anpr_ocr_md_2022_07_01_pytorch_lightning.ckpt",
                                              map_location=torch.device('cpu'),
                                              letters=letters,
                                              letters_max=len(letters) + 1,
                                              label_converter=label_converter,
                                              max_plate_length=max_plate_length)

modelv3_3 = NPOcrNetv3_3(letters,
                         letters_max=len(letters) + 1,
                         label_converter=label_converter,
                         max_text_len=max_plate_length)

state_dict_v3_2 = modelv3_2.state_dict()
state_dict_v3_3 = copy.deepcopy(state_dict_v3_2)
for key in state_dict_v3_2:
    if 'resnet' in key:
        key_v3_3 = key.replace('resnet', 'conv_nn')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
    elif 'gru' in key:
        key_v3_3 = key.replace('.gru.', '.rnn.')
        key_v3_3 = key_v3_3.replace('gru', 'recurrent_layer')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
        
modelv3_3.load_state_dict(state_dict_v3_3)
torch.save({"state_dict": modelv3_3.state_dict()}, RESULT_MODEL_PATH)

Downloading: "https://nomeroff.net.ua/models/ocr/md/torch/model_v3.1/anpr_ocr_md_2022_07_01_pytorch_lightning.ckpt" to /home/dmitroprobachay/.cache/torch/hub/checkpoints/anpr_ocr_md_2022_07_01_pytorch_lightning.ckpt


  0%|          | 0.00/26.5M [00:00<?, ?B/s]

None 512 32 50 200


## ru

In [68]:
DATASET_NAME = "ru"
VERSION = f"{datetime.now().strftime('%Y_%m_%d')}"

RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, 
                                 "data/models/", 
                                 'anpr_ocr_{}_{}.ckpt'.format(DATASET_NAME, VERSION))
letters = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "E", "H", "K", "M", "O",
           "P", "T", "X", "Y"]

max_plate_length = 9
label_converter = StrLabelConverter("".join(letters), max_plate_length)

modelv3_2 = NPOcrNetv3_2.load_from_checkpoint("https://nomeroff.net.ua/models/ocr/ru/torch/model_v3.1/anpr_ocr_ru_2022_03_28_pytorch_lightning.ckpt",
                                              map_location=torch.device('cpu'),
                                              letters=letters,
                                              letters_max=len(letters) + 1,
                                              label_converter=label_converter,
                                              max_plate_length=max_plate_length)

modelv3_3 = NPOcrNetv3_3(letters,
                         letters_max=len(letters) + 1,
                         label_converter=label_converter,
                         max_text_len=max_plate_length)

state_dict_v3_2 = modelv3_2.state_dict()
state_dict_v3_3 = copy.deepcopy(state_dict_v3_2)
for key in state_dict_v3_2:
    if 'resnet' in key:
        key_v3_3 = key.replace('resnet', 'conv_nn')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
    elif 'gru' in key:
        key_v3_3 = key.replace('.gru.', '.rnn.')
        key_v3_3 = key_v3_3.replace('gru', 'recurrent_layer')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
        
modelv3_3.load_state_dict(state_dict_v3_3)
torch.save({"state_dict": modelv3_3.state_dict()}, RESULT_MODEL_PATH)

Downloading: "https://nomeroff.net.ua/models/ocr/ru/torch/model_v3.1/anpr_ocr_ru_2022_03_28_pytorch_lightning.ckpt" to /home/dmitroprobachay/.cache/torch/hub/checkpoints/anpr_ocr_ru_2022_03_28_pytorch_lightning.ckpt


  0%|          | 0.00/13.3M [00:00<?, ?B/s]

None 512 32 50 200


## ru_military

In [69]:
DATASET_NAME = "ru_military"
VERSION = f"{datetime.now().strftime('%Y_%m_%d')}"

RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, 
                                 "data/models/", 
                                 'anpr_ocr_{}_{}.ckpt'.format(DATASET_NAME, VERSION))
letters = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "E", "H", "K", "M", "O",
           "P", "T", "X", "Y"]

max_plate_length = 8
label_converter = StrLabelConverter("".join(letters), max_plate_length)

modelv3_2 = NPOcrNetv3_2.load_from_checkpoint("https://nomeroff.net.ua/models/ocr/ru-military/torch/model_v3.1/anpr_ocr_ru_military_2022_03_28_pytorch_lightning.ckpt",
                                              map_location=torch.device('cpu'),
                                              letters=letters,
                                              letters_max=len(letters) + 1,
                                              label_converter=label_converter,
                                              max_plate_length=max_plate_length)

modelv3_3 = NPOcrNetv3_3(letters,
                         letters_max=len(letters) + 1,
                         label_converter=label_converter,
                         max_text_len=max_plate_length)

state_dict_v3_2 = modelv3_2.state_dict()
state_dict_v3_3 = copy.deepcopy(state_dict_v3_2)
for key in state_dict_v3_2:
    if 'resnet' in key:
        key_v3_3 = key.replace('resnet', 'conv_nn')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
    elif 'gru' in key:
        key_v3_3 = key.replace('.gru.', '.rnn.')
        key_v3_3 = key_v3_3.replace('gru', 'recurrent_layer')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
        
modelv3_3.load_state_dict(state_dict_v3_3)
torch.save({"state_dict": modelv3_3.state_dict()}, RESULT_MODEL_PATH)

None 512 32 50 200


## su

In [70]:
DATASET_NAME = "su"
VERSION = f"{datetime.now().strftime('%Y_%m_%d')}"

RESULT_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, 
                                 "data/models/", 
                                 'anpr_ocr_{}_{}.ckpt'.format(DATASET_NAME, VERSION))
letters = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
           'І', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И',
           'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У',
           'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Э', 'Ю', 'Я']

max_plate_length = 7
label_converter = StrLabelConverter("".join(letters), max_plate_length)

modelv3_2 = NPOcrNetv3_2.load_from_checkpoint("https://nomeroff.net.ua/models/ocr/su/torch/model_v3.1/anpr_ocr_su_2022_03_28_pytorch_lightning.ckpt",
                                              map_location=torch.device('cpu'),
                                              letters=letters,
                                              letters_max=len(letters) + 1,
                                              label_converter=label_converter,
                                              max_plate_length=max_plate_length)

modelv3_3 = NPOcrNetv3_3(letters,
                         letters_max=len(letters) + 1,
                         label_converter=label_converter,
                         max_text_len=max_plate_length)

state_dict_v3_2 = modelv3_2.state_dict()
state_dict_v3_3 = copy.deepcopy(state_dict_v3_2)
for key in state_dict_v3_2:
    if 'resnet' in key:
        key_v3_3 = key.replace('resnet', 'conv_nn')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
    elif 'gru' in key:
        key_v3_3 = key.replace('.gru.', '.rnn.')
        key_v3_3 = key_v3_3.replace('gru', 'recurrent_layer')
        state_dict_v3_3[key_v3_3] = state_dict_v3_3.pop(key)
        
modelv3_3.load_state_dict(state_dict_v3_3)
torch.save({"state_dict": modelv3_3.state_dict()}, RESULT_MODEL_PATH)

Downloading: "https://nomeroff.net.ua/models/ocr/su/torch/model_v3.1/anpr_ocr_su_2022_03_28_pytorch_lightning.ckpt" to /home/dmitroprobachay/.cache/torch/hub/checkpoints/anpr_ocr_su_2022_03_28_pytorch_lightning.ckpt


  0%|          | 0.00/13.3M [00:00<?, ?B/s]

None 512 32 50 200
